In [2]:

import optuna
import os
import numpy as np
import pandas as pd
import networkx as nx
import time
import torch
from matplotlib import pyplot as plt
from dask.distributed import Client, progress, wait
from IPython.display import display, HTML
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import f1_score

#import project_path
from src.util.generate_connected_graph import generate_connected_graph
from src.util.generate_lr_data import generate_low_rank_data
from src.util.generate_anomaly import generate_spatio_temporal_anomaly
# from src.algos.lr_stss import lr_stss
from src.algos.lr_stss_modified import lr_stss_modified
from src.algos.lr_sts import lr_sts_hard
from src.algos.lr_sss import lr_sss_hard
from src.algos.horpca_singleton import horpca_singleton
from dask.distributed import as_completed
from tqdm import tqdm
from sklearn.mixture import GaussianMixture
import math
from scipy.optimize import minimize_scalar
from scipy import stats
from scipy.stats import norm
from scipy.stats import multivariate_normal
import time


from src.algos.one_class_svm import one_class_svm
from sklearn.svm import OneClassSVM
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist
from src.util.LOF import lof
from src.algos.apply_lof import apply_lof


ModuleNotFoundError: No module named 'src'

In [ ]:
def t2m(X, m=1):
    """Matricisez the tensor X in the m'th mode.
    
    It is done by stacking fibers of mode m as column vectors.
    Order of the other modes follow cyclic order.
    ie ( I_m x I_(m+1). ... .I_N x I_0. ... I_(m-1) ).
    Args:
        X (np.ndarray): Tensor to be matricized
        m (int, optional): The mode whose fibers are stacked as vectors. Defaults to 1.
    Returns:
        M (np.ndarray): Matricized tensor.
    """
    n = X.shape
    if m>len(n) or m<1:
        raise ValueError(f"Invalid unfolding mode provided. m={m}, X shape:{n}")
    Xm = __roll_2_dim(X,m).ravel().reshape((n[m-1], int(np.prod(n)/n[m-1])))
    return Xm

def __roll_2_dim(X, m):
    n = X.shape; N = len(n)
    dest = np.arange(N)
    src = (np.arange(N) + (m-1))%N
    if isinstance(X, np.ndarray):
        return np.moveaxis(X, src, dest)
    elif isinstance(X, torch.Tensor):
        return torch.moveaxis(X, tuple(src), tuple(dest))

def convert_index(n,i,k):
    """Convert the index of an element of a tensor into it's k'th mode unfolding index

    Args:
        n (tuple): Tensor shape
        i (tuple): Tensor index
        k (int): Mode unfolding
    Returns:
        idx (tuple): Index of the element corresponding to the matricized tensor
    """
    if type(n) != tuple:
        raise TypeError("Dimension of the tensor, n is not a tuple")
    if type(i) != tuple:
        raise TypeError("index of the tensor element, i is not a tuple")
    if len(n) ==1:
        raise ValueError(f"The provided dimension n={n} is for the vector case")
    for j, i_ in enumerate(i):
        if i_>= n[j]:
            raise ValueError(f"Index i exceeds the dimension n in {j+1}'th mode")
    if k > len(n) or k<1:
        raise ValueError(f"Unfolding mode {k} is impossible for {len(n)}'th order tensor.")
    j=0
    idx = [i[k-1]]
    n_ = list(n)
    n_ = n_[k:] + n_[:k-1]
    i_ = list(i)
    i_ = i_[k:] + i_[:k-1]
    for p,i_k in enumerate(i_):
        j+= i_k*np.prod(n_[p+1:])
    idx.append(int(j))
    return tuple(idx)

In [ ]:
def m2t(Xm, dims, m=1):
    """Tensorizes the matrix obtained by t2m to its original state.

    Args:
        Xm (np.ndarray): Matrix
        dims (tuple): original dimensions of the tensor
        m (int): The mode for which the matricization was originally made with t2m. Defaults to 1.
    Returns:
        X (np.ndarray): Tensor
    """
    N = len(dims); 
    if m > N or m <1:
        raise ValueError(f"Invalid tensorization order m={m}, N={N}")
    
    old_dest = (np.arange(N) + (m-1))%N
    dims2 = tuple([dims[i] for i in old_dest])
    X = Xm.ravel().reshape(dims2)
    X = __unroll_from_dim(X, m)
    return X

def __unroll_from_dim(X, m):
    n = X.shape
    N = len(n)
    dest = (np.arange(N) + (m-1))%N
    src = np.arange(N)
    if isinstance(X, np.ndarray):
        return np.moveaxis(X, src, dest)
    elif isinstance(X, torch.Tensor):
        return torch.moveaxis(X, tuple(src), tuple(dest))


In [ ]:
## Control Variables
gt = 'grid'
graph_param = (8,5)
amp =1
window = 'boxcar'
distribution = 'bernoulli'
local_distribution = 'constant'
dims = (40,24,7,20)
ranks = (8,8,5,5)
NoG = 1
NoT = 10
time_m = 2
local_m = 1
lda_2 = 1000
psi = 23.467775089144254 # 20
maxit = 200
maxit_horpca = 200
## Independent variables
#NoA = np.array([25,50,125,500,1000])
#NoA = np.array([1500, 2000, 2500, 3000])

### Hyperparameters
#### LR-STSS Hyperparameters 
#lda_1 = 0.00012104 # 0.0003542734521053448 # np.logspace(1,-3,5)
#lda_l = 0.00016974 # 0.002062120382 # np.logspace(1,-3,5)
#lda_t = 0.00010008 # 0.00013502437304801454 # np.logspace(1,-3,5)


## LRSTSS

In [ ]:
lda_1 = np.array([0.0002570262901843806, 0.014143350419624914, 0.0003012786745130843, 0.005305415411080644, 0.0005731735080670809])
lda_l = np.array([0.046624757541464806, 0.0188934471162188, 0.003040251081845518, 0.0006354906259227154, 0.0019404405160872405])
lda_t = np.array([0.000223497225406953, 0.0001528854001880808, 0.004681368900651915, 0.06227466169063777, 0.00011704490142338973])

In [ ]:
it = 10
it_len = np.arange(0,it)
NoA = 300 #100
rad = np.arange(1,6) ## rad
radi = np.arange(len(rad))
dtn = 2
L_Output = np.zeros((it,len(rad),2))